In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob
from typing import Any, Dict, List, Tuple, Union
import pandas as pd

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 6

def train_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    out = [numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    idx = [numpy.dstack([scene['scene_idx']]) for scene in batch]
    return inp, idx
    
val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = True, collate_fn=train_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = True, collate_fn=test_collate, num_workers=0)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EncoderRNN(nn.Module):
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    def __init__(self,
                 input_size = 2,
                 embedding_size = 8,
                 hidden_size = 16):
        
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear(x))
        hidden = self.lstm(embedded, hidden)
        return hidden


class DecoderRNN(nn.Module):
    """Decoder Network."""
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    def __init__(self, embedding_size=8, hidden_size=16, output_size=2):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.linear1 = nn.Linear(output_size, embedding_size)
        self.lstm = nn.LSTMCell(embedding_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        embedded = F.relu(self.linear1(x))
        hidden = self.lstm(embedded, hidden)
        output = self.linear2(hidden[0])
        return output, hidden


In [5]:
from tqdm import tqdm_notebook as tqdm

def train(encoder, decoder, device, train_loader, encoder_optimizer, decoder_optimizer, epoch, log_interval=10000):    
    """referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting"""
    
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    criterion = nn.MSELoss()
    
    for i_batch, sample_batch in enumerate(train_loader):
        
        inp, out = sample_batch
        # preprocessing more ????
        inp = inp[:,0,:,:]
        out = out[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(inp.shape[0]):
            x_offset.append(inp[i][0][0].detach().clone())
            y_offset.append(inp[i][0][1].detach().clone())
    
        for j in range(inp.shape[0]):
            for i in range(inp.shape[1]):
                inp[j][i][0] = inp[j][i][0] - x_offset[j]
                inp[j][i][1] = inp[j][i][1] - y_offset[j]

        #outoput whatever
        for j in range(out.shape[0]):
            for i in range(out.shape[1]):
                out[j][i][0] = out[j][i][0] - x_offset[j]
                out[j][i][1] = out[j][i][1] - y_offset[j]
        
        _input, target = inp.to(device), out.to(device)
        
        encoder.train()
        decoder.train()
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        
        #encoder 
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        output_length = target.shape[1]
        feature_len = _input.shape[2]
        input_shape = _input.shape[2]
        
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
        
        loss = 0
        
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(target.shape).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Update loss
            loss += torch.sqrt(criterion(decoder_output[:, :2], target[:, di, :2]))

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        # Get average loss for pred_len
        loss = loss / 30

        # Backpropagate
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()
        
#         output = model(data)
#         loss = MSELoss(output, target)
        counter += 1
        iterator.set_postfix(loss=(loss.item()))

In [6]:
device = "cuda"
encoder = EncoderRNN(input_size=2)
decoder = DecoderRNN(output_size=2)

encoder = nn.DataParallel(encoder)
decoder = nn.DataParallel(decoder)

encoder.to(device)
decoder.to(device)

encoder_optimizer = torch.optim.Adam(encoder.parameters())
decoder_optimizer = torch.optim.Adam(decoder.parameters())

num_epoch = 18

for epoch in range(1, num_epoch + 1):
        train(encoder, decoder, device, val_loader, encoder_optimizer, decoder_optimizer, epoch)
#         predict(model, device, test_loader)

In [7]:
def infer_absolute(
        test_loader: torch.utils.data.DataLoader,
        encoder: EncoderRNN,
        decoder: DecoderRNN,
#         start_idx: int,
#         forecasted_save_dir: str,
#         model_utils: ModelUtils,
):
    """Infer function for non-map LSTM baselines and save the forecasted trajectories.
    
    referenced from official Argoverse forecasting code: https://github.com/jagjeet-singh/argoverse-forecasting
    
    Args:
        test_loader: DataLoader for the test set
        encoder: Encoder network instance
        decoder: Decoder network instance
        start_idx: start index for the current joblib batch
        forecasted_save_dir: Directory where forecasted trajectories are to be saved
        model_utils: ModelUtils instance

    """
    
    forecasted_trajectories = {}

    for i, (_input, idx) in enumerate(test_loader):
        
        _input = _input[:,0,:,:]
        
        #inp - inp[0] for all in whaetver
        x_offset = []
        y_offset = []
        for i in range(_input.shape[0]):
            x_offset.append(_input[i][0][0].detach().clone())
            y_offset.append(_input[i][0][1].detach().clone())
    
        for j in range(_input.shape[0]):
            for i in range(_input.shape[1]):
                _input[j][i][0] = _input[j][i][0] - x_offset[j]
                _input[j][i][1] = _input[j][i][1] - y_offset[j]

        _input = _input.to(device)

        # Set to eval mode
        encoder.eval()
        decoder.eval()

        # Encoder
        batch_size = _input.shape[0]
        input_length = _input.shape[1]
        input_shape = _input.shape[2]

        # Initialize encoder hidden state
        encoder_hidden = (torch.zeros(batch_size, encoder.module.hidden_size).to(device), 
                          torch.zeros(batch_size, encoder.module.hidden_size).to(device))
       
        # Encode observed trajectory
        for ei in range(input_length):
            encoder_input = _input[:, ei, :]
            encoder_hidden = encoder(encoder_input, encoder_hidden)

        # Initialize decoder input with last coordinate in encoder
        decoder_input = encoder_input[:, :2]

        # Initialize decoder hidden state as encoder hidden state
        decoder_hidden = encoder_hidden

        decoder_outputs = torch.zeros(
            (batch_size, 30, 2)).to(device)

        # Decode hidden state in future trajectory
        for di in range(30):
            decoder_output, decoder_hidden = decoder(decoder_input,
                                                     decoder_hidden)
            decoder_outputs[:, di, :] = decoder_output

            # Use own predictions as inputs at next step
            decoder_input = decoder_output

        for i in range(30):
            for j in range(batch_size):
                decoder_outputs[j,i,0] = decoder_outputs[j,i,0] + x_offset[j]
                decoder_outputs[j,i,1] = decoder_outputs[j,i,1] + y_offset[j]
            
                if (idx[j][0][0][0] in forecasted_trajectories):
                    forecasted_trajectories[idx[j][0][0][0]].append(decoder_outputs[j,i,:].tolist())
                else:
                    forecasted_trajectories[idx[j][0][0][0]] = [decoder_outputs[j,i,:].tolist()]
                
    return(forecasted_trajectories)

In [8]:
output = infer_absolute(test_loader, encoder, decoder)

In [9]:
import pandas as pd
df = pd.DataFrame.from_dict(output, orient='index')
df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
19303,"[1814.943359375, 420.9847717285156]","[1815.8985595703125, 421.6789855957031]","[1816.71484375, 422.3968811035156]","[1817.587646484375, 423.107666015625]","[1818.460205078125, 423.8227844238281]","[1819.3289794921875, 424.54473876953125]","[1820.1954345703125, 425.27142333984375]","[1821.059814453125, 426.001708984375]","[1821.9229736328125, 426.73492431640625]","[1822.7860107421875, 427.47088623046875]",...,"[1832.4710693359375, 435.7670593261719]","[1833.349609375, 436.5281982421875]","[1834.2166748046875, 437.2851257324219]","[1835.068603515625, 438.0359802246094]","[1835.9013671875, 438.77899169921875]","[1836.7113037109375, 439.5123596191406]","[1837.4949951171875, 440.23468017578125]","[1838.249755859375, 440.9446105957031]","[1838.9732666015625, 441.64129638671875]","[1839.664306640625, 442.32403564453125]"
7696,"[1977.822998046875, 564.3927001953125]","[1976.457275390625, 563.3143310546875]","[1975.3387451171875, 562.2564697265625]","[1974.193603515625, 561.2669677734375]","[1973.052490234375, 560.275634765625]","[1971.893310546875, 559.269775390625]","[1970.7198486328125, 558.2532958984375]","[1969.5362548828125, 557.2296142578125]","[1968.3492431640625, 556.2039794921875]","[1967.16552734375, 555.181640625]",...,"[1954.59912109375, 544.2119140625]","[1953.439697265625, 543.1799926757812]","[1952.28125, 542.148681640625]","[1951.129150390625, 541.1261596679688]","[1949.9879150390625, 540.1203002929688]","[1948.8597412109375, 539.1380004882812]","[1947.74462890625, 538.1845092773438]","[1946.642578125, 537.2637939453125]","[1945.5562744140625, 536.3800659179688]","[1944.494384765625, 535.5386962890625]"
5171,"[1669.0052490234375, 512.28466796875]","[1669.713134765625, 512.775634765625]","[1670.349365234375, 513.3515625]","[1671.0477294921875, 513.9194946289062]","[1671.731201171875, 514.4828491210938]","[1672.412353515625, 515.04833984375]","[1673.0947265625, 515.6165161132812]","[1673.78076171875, 516.1884155273438]","[1674.471923828125, 516.7642211914062]","[1675.1688232421875, 517.3441162109375]",...,"[1683.1470947265625, 523.9620971679688]","[1683.8768310546875, 524.5845947265625]","[1684.6004638671875, 525.210205078125]","[1685.316162109375, 525.8388061523438]","[1686.021728515625, 526.4701538085938]","[1686.71533203125, 527.1041259765625]","[1687.3955078125, 527.7406005859375]","[1688.0604248046875, 528.3794555664062]","[1688.7088623046875, 529.0203857421875]","[1689.33984375, 529.6632080078125]"
18512,"[2015.711669921875, 598.412841796875]","[2014.5611572265625, 597.580322265625]","[2013.525390625, 596.6469116210938]","[2012.5308837890625, 595.8356323242188]","[2011.54833984375, 595.0150146484375]","[2010.5673828125, 594.1873168945312]","[2009.578857421875, 593.3470458984375]","[2008.5797119140625, 592.4933471679688]","[2007.5689697265625, 591.6260986328125]","[2006.5478515625, 590.74658203125]",...,"[1995.350341796875, 580.8612670898438]","[1994.3770751953125, 579.986083984375]","[1993.41552734375, 579.1211547851562]","[1992.466552734375, 578.2684326171875]","[1991.531005859375, 577.4300537109375]","[1990.609375, 576.60791015625]","[1989.7017822265625, 575.8037719726562]","[1988.80810546875, 575.0194091796875]","[1987.928466796875, 574.2561645507812]","[1987.0631103515625, 573.5155639648438]"
13141,"[1784.9014892578125, 406.2430114746094]","[1784.2410888671875, 405.7900085449219]","[1783.433349609375, 405.0364685058594]","[1782.6871337890625, 404.4041442871094]","[1781.9423828125, 403.7747497558594]","[1781.2147216796875, 403.1495666503906]","[1780.4935302734375, 402.52484130859375]","[1779.7757568359375, 401.8985595703125]","[1779.05810546875, 401.2688293457031]","[1778.33837890625, 400.6341857910156]",...,"[1770.26513671875, 393.29034423828125]","[1769.54248046875, 392.611328125]","[1768.826171875, 391.9352722167969]","[1768.11669921875, 391.26312255859375]","[1767.4146728515625, 390.595703125]","[1766.7200927734375, 389.933837890625]","[1766.0334

In [10]:
for i in range(30):
    df[['v{}'.format((i*2)+1), 'v{}'.format((i*2)+2)]] = pd.DataFrame(df.get(i).tolist(), index=df.index)

In [11]:
import numpy as np
dropped_cols = list(np.arange(30))
df2 = df.drop(dropped_cols, axis=1)
df2.index.name = 'ID'
df2

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v51,v52,v53,v54,v55,v56,v57,v58,v59,v60
ID,,,,,,,,,,,,,,,,,,,,,
19303,1814.943359,420.984772,1815.898560,421.678986,1816.714844,422.396881,1817.587646,423.107666,1818.460205,423.822784,...,1836.711304,439.512360,1837.494995,440.234680,1838.249756,440.944611,1838.973267,441.641296,1839.664307,442.324036
7696,1977.822998,564.392700,1976.457275,563.314331,1975.338745,562.256470,1974.193604,561.266968,1973.052490,560.275635,...,1948.859741,539.138000,1947.744629,538.184509,1946.642578,537.263794,1945.556274,536.380066,1944.494385,535.538696
5171,1669.005249,512.284668,1669.713135,512.775635,1670.349365,513.351562,1671.047729,513.919495,1671.731201,514.482849,...,1686.715332,527.104126,1687.395508,527.740601,1688.060425,528.379456,1688.708862,529.020386,1689.339844,529.663208
18512,2015.711670,598.412842,2014.561157,597.580322,2013.525391,596.646912,2012.530884,595.835632,2011.548340,595.015015,...,1990.609375,576.607910,1989.701782,575.803772,1988.808105,575.019409,1987.928467,574.256165,1987.063110,573.515564
13141,1784.901489,406.243011,1784.241089,405.790009,1783.433350,405.036469,1782.687134,404.404144,1781.942383,403.774750,...,1766.720093,389.933838,1766.033447,389.278137,1765.354370,388.629242,1764.682983,387.987701,1764.019043,387.354004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34856,587.935242,1524.833740,587.718140,1525.435669,587.598083,1525.953369,587.517334,1526.482178,587.447937,1527.013184,...,586.227539,1536.716919,586.132629,1537.091064,586.032532,1537.459106,585.927368,1537.821411,585.817322,1538.178833
29690,1776.704224,393.998444,1776.831421,394.479584,1777.219604,394.987335,1777.564941,395.483612,1777.891357,395.979889,...,1782.440308,407.341644,1782.538574,407.856323,1782.619507,408.362457,1782.681519,408.859589,1782.722778,409.347321
19350,729.732483,1530.842529,729.911377,1529.625610,729.846558,1528.601318,729.788147,1527.508789,729.747681,1526.439209,...,729.259216,1504.744141,729.262024,1503.729004,729.267761,1502.716064,729.275635,1501.705811,729.284485,1500.699097


In [12]:
df2.to_csv("outputs18epb6.csv", index=True)